# Импорт библиотек

In [5]:
import pandas as pd
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import numpy as np
import torchvision
import random
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import torch.onnx

In [6]:
DATA_MODES = ['train', 'val', 'test']
RESCALE_SIZE = 224
EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 0.001

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Модели

## Модель `create_nn_v2`

In [7]:
data_dir = './data/classed_type_train/'
test_dir = './data/test2/'

In [8]:
#Набор трансформаций для обучающей выборки
train_transforms = transforms.Compose([transforms.CenterCrop(224), 
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomVerticalFlip(p=0.5),
                                       transforms.RandomRotation(degrees=(45, 90)),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])
                                       ])

In [9]:
#Набор трансформаций для валидационной выборки
valid_transforms = transforms.Compose([transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           [0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225])
                                       ])

In [10]:
#Набор трансформаций для тестовой выборки
test_transforms = transforms.Compose([transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(
                                          [0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225])
                                      ])

In [11]:
#Объявляю функцию загрузчика с аргументами:
#Директория, Размер батча, Набор трансформаций для тренировочной выборки,
#Набор трансформаций для валидационной выборки, Размер валидационной выборки
def load_split_train_valid(datadir,
                           batch_size,
                           train_transforms,
                           valid_transforms,
                           valid_size): 
    
    #Загрузчик для тренировочной выборки
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    #Загрузчик для тренировочной выборки
    test_data = datasets.ImageFolder(datadir,
                    transform=valid_transforms)
    #Код для разделения на трейн и тест в указанном соотношении
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=batch_size)
    return trainloader, testloader

In [12]:
#Получаю тренировочный и валидационный генераторы
train_loader, val_loader = load_split_train_valid(datadir=data_dir,
                           batch_size=BATCH_SIZE,
                           train_transforms=train_transforms,
                           valid_transforms=valid_transforms,
                           valid_size = .2)
#Проверяю результаты работы генераторов
print(train_loader.dataset.classes)
print(val_loader.dataset.classes)
print(len(train_loader))
print(len(val_loader))

['ER', 'NR']
['ER', 'NR']
163
41


In [13]:
class ConvNet(nn.Module): 
        def __init__(self):
            super(ConvNet, self).__init__()
            self.c1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
            self.c2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
            self.c3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
            self.fc1 = nn.Linear(in_features=6272, out_features=1024)
            self.fc2 = nn.Linear(in_features=1024, out_features=512)
            self.fc3 = nn.Linear(in_features=512, out_features=256)
            self.fc4 = nn.Linear(in_features=256, out_features=6)
        
        def forward(self, x):
            x = F.relu(F.max_pool2d(self.c1(x), 3))
            x = F.relu(F.max_pool2d(self.c2(x), 3))
            x = F.relu(F.max_pool2d(self.c3(x), 3))
            x = x.view(x.size(0), -1)
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            x = self.fc4(x)

            return F.log_softmax(x, dim=1)

In [14]:
net = ConvNet()
print(net)
#net = net.cuda()
net = net.to(device)

ConvNet(
  (c1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (c2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (c3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=6272, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=6, bias=True)
)


In [15]:
#Задаю функцию потерь
criterion = nn.CrossEntropyLoss()
#Задаю оптимизатор
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)

total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in tqdm(range(EPOCHS)):
    for i, (images, labels) in enumerate(train_loader):
    #Прямой запуск
        images=images.to(device)
        labels=labels.to(device)
            
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        #Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
            .format(epoch + 1, EPOCHS, i + 1, total_step, loss.item(),
                                  (correct / total) * 100))


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Epoch [1/10], Step [100/163], Loss: 0.6126, Accuracy: 62.50%


 10%|████████▏                                                                         | 1/10 [01:47<16:06, 107.38s/it]

Epoch [2/10], Step [100/163], Loss: 0.2487, Accuracy: 89.06%


 20%|████████████████▌                                                                  | 2/10 [02:33<09:31, 71.38s/it]

...

 80%|██████████████████████████████████████████████████████████████████▍                | 8/10 [07:20<01:40, 50.11s/it]

Epoch [9/10], Step [100/163], Loss: 0.0138, Accuracy: 98.44%


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [08:10<00:50, 50.18s/it]

Epoch [10/10], Step [100/163], Loss: 0.0008, Accuracy: 100.00%


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [08:59<00:00, 53.94s/it]


In [16]:
#Проверка на валидационной выборке
net.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in tqdm(val_loader):
        images=images.to(device)
        labels=labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the test images: {} %'
              .format((correct / total) * 100))

100%|██████████████████████████████████████████████████████████████████████████████████| 41/41 [00:25<00:00,  1.59it/s]

Test Accuracy of the model on the test images: 99.38295410721172 %


In [35]:
test_data = datasets.ImageFolder(test_dir,
                transform=test_transforms)

test_loader = DataLoader(test_data, shuffle=False, batch_size=BATCH_SIZE)
filenames = [line[0].split('\\')[1].split('.')[0] for line in test_data.imgs]
pred2class = [0 if n == 'ER' else 1 for n in train_loader.dataset.classes]
dict_pred = defaultdict(list)
dict_pred["id"] = filenames

In [36]:
%%time
net.eval()
with torch.no_grad():
    for i, (img, name) in enumerate(iter(test_loader)):
        img = img.to(device)
        outputs = net(img)
        _, predicted = torch.max(outputs.data, 1)
        predicted = list(map(int, predicted.cpu()))
        dict_pred["classification_predictions"].extend(predicted)

Wall time: 1min


In [37]:
print(len(dict_pred['id']))
print(len(dict_pred['classification_predictions']))

16564
16564


In [38]:
data_frame = pd.DataFrame(dict_pred, columns=["id", "classification_predictions"])
data_frame.to_csv('./submissions/submission_tryCNN_cls.csv', index=False, header=True)